In [1]:
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from string import punctuation
from collections import Counter

### Json model: 
```json
{
    "collections":[
      {
         "type":"detective",
         "url":"url",
         "text":"text" # выборка 30-50kb
      },
      {
          ...
      }
    ]
}
```

In [2]:
detective = './dataset/detective.json'
def read_file(file_path: str) -> str:
    """читает файл"""
    with open(file_path, "r") as f:
        content = f.read()
    return content

In [3]:
def read_stopwords(file_path: str) -> list:
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

additional_stopwords = read_stopwords('./dataset/stopwords.txt')

In [4]:
file_contents = read_file(detective)
# print(file_contents)
parsed_json = json.loads(file_contents)

In [5]:
def tokenize_ru(file_text: str) -> list:
    """удаляет из текста символы пунктуации, стоп-слова"""
    # применим токенизацию
    tokens = word_tokenize(file_text)

    # удаляем пунктуационные символы
    tokens = [i for i in tokens if (i not in punctuation)]
    
    # чистим слова
    tokens = [i.lower().replace("«", "").replace("»", "").replace("`", "").replace("'", "").replace("\\", "") for i in tokens]
    tokens = list(filter(None, tokens))
    
    # удаляем стоп-слова
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...', '.', '-'])
    stop_words.extend(additional_stopwords)
    tokens = [i for i in tokens if (i not in stop_words)]

    return tokens

In [6]:
filtered_detective = tokenize_ru(parsed_json['collections'][0]['text'])
text = nltk.Text(filtered_detective)

histogram_words = Counter(filtered_detective) # type: collections.Counter
# filtered_detective
# print((histogram_words))
fdist = FreqDist(text)
# fdist
# fdist.most_common(10)

In [7]:
def normalize_dict(input_dict: dict) -> dict:
    """минимаксная нормализация словаря"""
    new_dict = {}
    min_val = min(input_dict.values())
    max_val = max(input_dict.values())
    for key, value in input_dict.items():
        try:
            new_dict[key] = (input_dict[key] - min_val) / (max_val - min_val)
        except ZeroDivisionError:
            new_dict[key] = 0
    return new_dict

In [8]:
normalized_dict = normalize_dict(histogram_words)

# normalized_dict

In [9]:
def dict_without_values_eq_zero(input_dict: dict) -> dict:
    """словарь без значений равных нулю"""
    return {k:v for k,v in input_dict.items() if v != 0}

dict_without_zero = dict_without_values_eq_zero(normalized_dict)
# dict_without_zero

In [10]:
feature_dict: dict[int, dict] = {}

for index, item in enumerate(parsed_json['collections'], start=1):
    # обработка текста
    tokenized_item = tokenize_ru(item['text']) # разбиваем по словам
    item_histogram = dict(Counter(tokenized_item)) # строим гистограмму слов
    normalized_histogram = normalize_dict(item_histogram) 
    dict_without_zero = dict_without_values_eq_zero(normalized_histogram) # удаляем слова по одному вхождению 
    dict_without_zero = {k: v for k, v in sorted(dict_without_zero.items(), key=lambda item: item[1], reverse=True)}
    # cобираем словарь фичей dict[index] = dict(feature)
    feature_dict[index] = dict_without_zero
    # normalized_dict = {}
    # print(index, len(feature_dict[index]))
    # print()
    
# feature_dict

In [18]:
all_words = []
for it in feature_dict.values():
    # print((it.keys()))
    all_words.append(list(it.keys()))
for word in all_words:
    print(word)
flattened_list = [item for sublist in all for item in sublist]

# vocabulary = set(all_words)


['книги', 'король', 'книгу', 'короля', 'ридер', 'обливии', 'понял', 'буду', 'кошка', 'блок', 'ваше', 'мир', 'редактор', 'фбр', 'глаза', 'второй', 'своей', 'конец', 'зал', 'обуса', 'радио', 'вилсон', 'разные', 'назад', 'читатель', 'затем', 'мог', 'книге', 'возможность', 'найти', 'место', 'почему', 'пришлось', 'время', 'мигом', 'одно', 'бюро', 'стороны', 'хочешь', 'сигару', 'лицо', 'сегодня', 'вице-президент', 'компании', 'ясно', 'встречу', 'держи', 'зато', 'именно', 'дело', 'скроллз', 'таверны', 'серая', 'стол', 'лапу', 'абсента', 'астарта', 'астарту', 'огромный', 'брутрих', 'скуримии', 'эрзац', 'дня', 'сидел', 'интересно', 'нос', 'полгода', 'своего', 'королева', 'королевы', 'мной', 'настоящего', 'прикончить', 'величество', 'готов', 'площадь', 'столь', 'тысячи', 'лет', 'давно', 'стали', 'книга', 'переживает', 'прочих', 'выбор', 'написали', 'специально', 'любой', 'приключения', 'шкуре', 'причем', 'ощущения', 'настолько', 'жизни', 'тело', 'редактора', 'специалиста', 'багов', 'читателя', '